In [ ]:
# from mv_dts import MusicVideoDatasetV2
# T_RES = 10
# dts = MusicVideoDatasetV2("test", frames=10, duration=10, t_res=2)

# errors = {}
# labels = {i: 0 for i in range(T_RES)}

# for i in range(len(dts)):
#     try:
#         b = dts[i]
#         print(i, b["labels"].item())
#         labels[b["labels"].item()] += 1
#     except Exception as e:
#         print("exception", e)
#         errors[i] = e
#         continue
# # _ = dts[random.randrange(0, len(dts))]
# total = sum([v for k, v in labels.items()])
# labels = {k: v / total for k, v in labels.items()}
# labels

In [ ]:
import numpy as np
from mv_dts import MusicVideoDatasetV3
T_RES = 3
dts = MusicVideoDatasetV3("test", frames=20, duration=10, t_res=T_RES, with_audio=True, debug=True, offset_ratio=0.3)
np.random.seed(1019)
idx = 37
print(len(dts))

In [ ]:
import os
import matplotlib.pyplot as plt
b = dts.__getitem__(idx, True)
print(b["labels"].item())
frames = b["frames"].permute(2, 0, 3, 1).flatten(1, 2)
print(frames.shape)
plt.figure(figsize=(120, 12))
plt.imshow(frames)
plt.show()
plt.close()
idx = 1 + idx
os.system(f"ffmpeg -hide_banner  -loglevel error -y -ss {b['timestamps'][0]} -t {int(b['timestamps'][2]-b['timestamps'][0])} -i  {b['video_path']} sample.mp4")
import IPython
IPython.display.Audio(b["audio"], rate=44100, normalize=True)

In [ ]:
b.keys()

In [ ]:
b["audio"].shape

In [ ]:
import torchvision
import numpy as np
from decord import VideoReader, cpu


def load_video(video_path):
    vr = VideoReader(video_path, ctx=cpu(0))
    total_frame_num = len(vr)
    uniform_sampled_frames = np.linspace(0, total_frame_num - 1, 15, dtype=int)
    frame_idx = uniform_sampled_frames.tolist()
    spare_frames = vr.get_batch(frame_idx).asnumpy()
    return spare_frames


def fetch_frames(video_path, duration, offset=0, num_frames=10, transform=(lambda x: x)):
    frames = torchvision.io.read_video(video_path, offset, offset + duration, 'sec', 'TCHW')[0]
    if (num_frames > 1):
        stride = (frames.shape[0] - 1) / (num_frames - 1)  # number of video frames
    else:
        stride = 0
    frames = frames[[int(i * stride) for i in range(num_frames)]]
    assert frames.shape[0] == num_frames, "number of frames is not correct"
    frames = transform(frames)
    return frames


frames_1 = load_video("out_2.mp4")
frames_2 = fetch_frames("out_2.mp4", duration=30, num_frames=15)

In [ ]:
import matplotlib.pyplot as plt

for i in frames_1:
    plt.imshow(i)
    plt.show()
    plt.close()

In [ ]:
for i in frames_2.permute(0, 2, 3, 1):
    plt.imshow(i)
    plt.show()
    plt.close()

In [ ]:
frames_1.shape

In [ ]:
image_processor.preprocess(video, return_tensors="pt")["pixel_values"].half().cuda()

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "9"
from llavavid.model.builder import load_pretrained_model


def get_model_name_from_path(model_path):
    model_path = model_path.strip("/")
    model_paths = model_path.split("/")
    if model_paths[-1].startswith('checkpoint-'):
        return model_paths[-2] + "_" + model_paths[-1]
    else:
        return model_paths[-1]


tokenizer, model, image_processor, context_len = load_pretrained_model(
    "lmms-lab/LLaVA-NeXT-Video-7B-DPO", None, get_model_name_from_path("lmms-lab/LLaVA-NeXT-Video-7B-DPO")
)

In [ ]:
video_1 = image_processor.preprocess(frames_1, return_tensors="pt")["pixel_values"].half().cuda()

In [ ]:
video_2 = image_processor.preprocess(frames_1.transpose((0, 3, 1, 2)), return_tensors="pt")["pixel_values"].half().cuda()

In [ ]:
video_1.shape, video_2.shape

In [ ]:
False in (video_1 == video_2)

In [ ]:
import torch
(frames_2.permute(0, 2, 3, 1) - torch.from_numpy(frames_1)).float().abs().min()

In [ ]:
video_1 == video_2

In [ ]:
import os
import cv2
import math
from scenedetect import detect, AdaptiveDetector, split_video_ffmpeg
SCENE_CUT_DIR = "scene_cut"
FRAME_CUT_DIR = "frame_cut"

scene_list = detect("/scratch4/users/od/YTCharts/videos/-KKMiVcuwJk.mp4", AdaptiveDetector())
# scene_list = detect('/scratch4/users/od/YTCharts/v1/test/--Sh1ABUSas.mp4', AdaptiveDetector())
len(scene_list)
# save each scene cut
# split_video_ffmpeg('/scratch4/users/od/YTCharts/v1/test/6R4JLBC3vqc.mp4', scene_list, output_dir=SCENE_CUT_DIR)


# MIN_CUTS = 15
# cut_frame_indices = []
# if (len(scene_list) >= MIN_CUTS):
#     for (a, b) in scene_list:
#         cut_frame_indices.append((a.frame_num + b.frame_num) // 2)
# else:
#     # count ratio
#     per_scene_frame_num = []
#     for (a, b) in scene_list:
#         per_scene_frame_num.append(b.frame_num - a.frame_num)
#     total = sum(per_scene_frame_num)
#     ratio = [
#         max(1, round(MIN_CUTS * (frame_num / total)))
#         for frame_num in per_scene_frame_num
#     ]
#     ratio[-1] += max(MIN_CUTS - sum(ratio), 0)

#     for r, f, (a, b) in zip(ratio, per_scene_frame_num, scene_list):
#         hop = f // (r + 1)
#         for i in range(r):
#             cut_frame_indices.append(a.frame_num + (1 + i) * hop)

# print(cut_frame_indices)

# os.makedirs(FRAME_CUT_DIR, exist_ok=True)
# cap = cv2.VideoCapture("./original.mp4")
# j = 0
# for i in range(round(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
#     ret, frame = cap.read()
#     if (i in cut_frame_indices):
#         cv2.imwrite(os.path.join(FRAME_CUT_DIR, f"{j}.png"), frame)
#         j += 1
# cap.release()

In [ ]:
import torch
import matplotlib.pyplot as plt
lr_sched = torch.optim.lr_scheduler.OneCycleLR(torch.optim.AdamW([torch.tensor([0, 0, 0])], lr=0.1), 1e-4, total_steps=100, epochs=None, steps_per_epoch=None, pct_start=0.3, anneal_strategy='cos',
                                               cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=25.0, final_div_factor=10000.0, three_phase=False, last_epoch=-1, verbose='deprecated')

lr = []
for i in range(100):
    lr_sched.step()
    lr.append(lr_sched.get_last_lr())

plt.plot(range(100), lr)
plt.show()

In [ ]:
import json
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

scenes_list = []
for file in glob("/scratch4/users/od/YTCharts/v1/*/*.json"):
    with open(file, "r") as f:
        scenes = json.load(f)["scenes"]
        if (scenes == 10):
            print(file)
        scenes_list.append(scenes)

In [ ]:
counts, bins = np.histogram(scenes_list, bins=100, range=(0, 300))
plt.stairs(counts, bins)
plt.xlabel("scene cuts")
plt.ylabel("count")
plt.title("YTCharts Scene Cut Histogram")

In [ ]:
import os

import torch.utils
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["PYTHONWARNINGS"] = "ignore"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"


import os
import sys
import math
import torch
import random
import argparse
import numpy as np
from tqdm import tqdm
from datetime import datetime
from transformers import AutoConfig
from peft import LoraConfig, TaskType, PeftModel, get_peft_model

from sklearn.metrics import accuracy_score

from llavavid.conversation import conv_templates, SeparatorStyle
from llavavid.mm_utils import tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria
from llavavid.constants import IGNORE_INDEX, IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN


def load_pretrained_model(model_path, model_base, model_name, load_8bit=False, load_4bit=False, device_map="auto", overwrite_config=None):
    import os
    import torch

    from transformers import AutoTokenizer, AutoConfig, BitsAndBytesConfig
    from llavavid.model import LlavaLlamaForCausalLM
    from llavavid.constants import DEFAULT_IMAGE_PATCH_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
    kwargs = {"device_map": device_map}

    # import pdb;pdb.set_trace()
    if load_8bit:
        kwargs["load_in_8bit"] = True
    elif load_4bit:
        kwargs["load_in_4bit"] = True
        kwargs["quantization_config"] = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4")
    else:
        kwargs["torch_dtype"] = torch.float16

  # this may be mm projector only
    print("Loading LLaVA from base model...")

    tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
    cfg_pretrained = AutoConfig.from_pretrained(model_path)
    if overwrite_config is not None:
        print(f"Overwriting config with {overwrite_config}")
        for k, v in overwrite_config.items():
            setattr(cfg_pretrained, k, v)
    print(cfg_pretrained)
    model = LlavaLlamaForCausalLM.from_pretrained(model_path, low_cpu_mem_usage=True, config=cfg_pretrained, **kwargs)

    image_processor = None

    assert "llava" in model_name.lower(), "Only LLaVA models are supported for video chatbot."
    mm_use_im_start_end = getattr(model.config, "mm_use_im_start_end", False)
    mm_use_im_patch_token = getattr(model.config, "mm_use_im_patch_token", True)
    if mm_use_im_patch_token:
        tokenizer.add_tokens([DEFAULT_IMAGE_PATCH_TOKEN], special_tokens=True)
    if mm_use_im_start_end:
        tokenizer.add_tokens([DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN], special_tokens=True)
    model.resize_token_embeddings(len(tokenizer))

    vision_tower = model.get_vision_tower()
    if not vision_tower.is_loaded:
        vision_tower.load_model(device_map=device_map)
    vision_tower.to(device=model.device, dtype=model.dtype)
    image_processor = vision_tower.image_processor

    if hasattr(model.config, "max_sequence_length"):
        context_len = model.config.max_sequence_length
    else:
        context_len = 2048

    return tokenizer, model, image_processor, context_len


CONV_MODE = "vicuna_v1"
MODEL_PATH = "lmms-lab/LLaVA-NeXT-Video-7B-DPO"


# Initialize the model
model_name = get_model_name_from_path(MODEL_PATH)

# Set model configuration parameters if they exist
overwrite_config = {}
overwrite_config["mm_resampler_type"] = "spatial_pool"
overwrite_config["mm_spatial_pool_stride"] = 2
overwrite_config["mm_spatial_pool_out_channels"] = 1024
overwrite_config["mm_spatial_pool_mode"] = "average"
overwrite_config["patchify_video_feature"] = False

cfg_pretrained = AutoConfig.from_pretrained(MODEL_PATH)

if "224" in cfg_pretrained.mm_vision_tower:
    # suppose the length of text tokens is around 1000, from bo's report
    least_token_number = 10 * (16 // 2)**2 + 1000
else:
    least_token_number = 10 * (24 // 2)**2 + 1000

scaling_factor = math.ceil(least_token_number / 4096)

if scaling_factor >= 2:
    if "mistral" not in cfg_pretrained._name_or_path.lower() and "7b" in cfg_pretrained._name_or_path.lower():
        print(float(scaling_factor))
        overwrite_config["rope_scaling"] = {"factor": float(scaling_factor), "type": "linear"}
    overwrite_config["max_sequence_length"] = 4096 * scaling_factor
    overwrite_config["tokenizer_model_max_length"] = 4096 * scaling_factor

# Load model with new configuration
_, model, _, _ = load_pretrained_model(
    MODEL_PATH,
    None,
    model_name,
    False,
    overwrite_config=overwrite_config,
    device_map="cpu"
)

In [ ]:

# torch.nn.init.kaiming_uniform_ = lambda x, *args, **kwargs: x
# torch.nn.init.uniform_ = lambda x, *args, **kwargs: x
with torch.device("meta"):
    model = PeftModel.from_pretrained(
        model,
        "logs/0729_164405/v4_e47b624_loss=0.87",
        is_trainable=False,
        low_cpu_mem_usage=True,
        init_lora_weights=False
    )

In [ ]:
import numpy as np
import pickle

with open("data.pickle", "rb") as f:
    data = pickle.load(f)

In [ ]:
import matplotlib.pyplot as plt

labels = data["eval_labels"]
plt.hist(labels, bins=3, weights=np.ones(len(labels)) / len(labels))
plt.show()
plt.close()
probs = data["eval_probs"]
preds = probs.argmax(axis=-1)
plt.hist(preds, bins=3, weights=np.ones(len(preds)) / len(preds))
plt.show()
plt.close()
print("Accuracy:", (labels == preds).astype(int).mean())
print("Loss:", -np.log(probs[range(len(labels)), labels] + 1e-4).mean())

In [ ]:
labels.shape

In [ ]:
max(preds)

In [ ]:
data

In [ ]:
import torch
import pickle

with open("data.pickle", "rb") as f:
    in_data = pickle.load(f)["eval_records"]


tr_data = torch.load("logs/0802_044652/v4_e4b0_eval/aux.pt")["eval_records"]

In [ ]:
in_data = sorted(in_data, key=lambda x: x["idx"])
tr_data = sorted(tr_data, key=lambda x: x["idx"])

In [ ]:
for a, b in zip(in_data, tr_data):
    if (not a["video_path"] == b["video_path"]):
        print("Video Path", a["video_path"], b["video_path"])
    if (not a["idx"] == b["idx"]):
        print("Video IDX:", a["idx"], b["idx"])
    if (not ((a["timestamps"] - b["timestamps"]).abs().sum() == 0)):
        print("TimeStamps:", a["timestamps"], b["timestamps"])

In [ ]:
import torch

aux = torch.load("logs/0802_044652/v4_e4b0_eval/aux.pt")
adapt = torch.load("logs/0802_044652/v4_e4b0_eval/adapter_model.bin")
out = torch.load("logs/0802_044652/v4_e4b0_eval/out.pt")

In [ ]:
print(
    ((aux["t_proj"]["proj3.weight"].cpu() - out["t_proj.proj3.weight"].cpu().half())).abs().float().sum(),
    ((aux["embed_tokens"]["weight"].cpu() - out["model.base_model.model.model.embed_tokens.weight"].cpu().half())).abs().float().sum(),
    ((aux["lm_head"]["weight"].cpu() - out["model.base_model.model.lm_head.weight"].cpu().half())).abs().float().sum(),
)

In [ ]:
sum([
    ((adapt[f"base_model.model.model.layers.{i}.self_attn.{v}_proj.lora_{t}.weight"].cpu() -
     out[f"model.base_model.model.model.layers.{i}.self_attn.{v}_proj.lora_{t}.default.weight"].cpu().half())).abs().float().sum()
    for i in range(32)
    for t in ["A", "B"]
    for v in ["q", "k", "v", "o"]
])

In [ ]:
sum([
    ((adapt[f"base_model.model.model.layers.{i}.mlp.{v}_proj.lora_{t}.weight"].cpu() -
     out[f"model.base_model.model.model.layers.{i}.mlp.{v}_proj.lora_{t}.default.weight"].cpu().half())).abs().float().sum()
    for i in range(32)
    for t in ["A", "B"]
    for v in ["gate", "down", "up"]
])